In [1]:
from scipy.optimize import curve_fit
def calibration(x, a, b, c,d):
    x0 = x

    fout = np.exp(a*(x0)+b)+c
    fout = np.minimum(np.ones(x.shape)*100*d,fout)
    return np.maximum(np.zeros(x.shape),fout)

In [2]:
import pandas as pd
import numpy as np
cols = [0,1,2,4]#,4,5]
folds=4
nbranches=5
cells_list = ['Neuoplatic',"Inflammatory","Connective","Epithelial"]
metric = ['MAE',"Pearson","Spearman","R2","AUROC"]
R = np.random.random(size=(folds-1,len(cols),len(metric)))
table = pd.DataFrame()
vpath = 'BRCA_val/fold_'
tpath = 'BRCA_train/fold_'

TAG = 'ALBRT_PanNuke'

from keras.applications.xception import Xception
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers.core import Dense
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizer_v2.adam import Adam
import keras
from scipy.stats import pearsonr,spearmanr
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn import metrics
import matplotlib.pyplot as plt


for fold in range(1,folds):
    Xtr,Ytr=np.array([]),np.array([])
    Xts,Yts = np.array([]),np.array([])
    import gc;gc.collect()
    F = np.load(vpath+str(fold)+'.npz')
    if Yts.size==0:
        Xts,Yts = F['X'],F['Y'][:,0,cols]
    else:
        Xts,Yts = np.vstack((Xts,F['X'])),np.vstack((Yts,F['Y']))

    Xts,Yts = Xts.astype('float32')*255.0,Yts.astype('float32')

    print("Number of test example",Xts.shape, Yts.shape)

    base_model = Xception(
        include_top = False,
        weights = "imagenet",
        input_shape = None
    )

    x = base_model.output
    x = GlobalAveragePooling2D(name='pool')(x)
    x = Dense(1024,name='fc1')(x)
    branches = [Dense(len(cols),name='T'+str(i))(x) for i in range(nbranches)]

    class CustomModel(keras.Model):
        pass
    
    model = CustomModel(base_model.input, branches)
    model.compile(
        optimizer = Adam()
    )
    
    model.load_weights('weights/weights_'+TAG+str(fold)+'.hdf5')

    Ypred=model.predict(Xts)
    results = []
    for j in range(len(cols)):
        target = Yts[:,j]; pred = Ypred[0][:, j];
        tc = target.copy()
        tc[tc<=0]=0
        tc[tc>0]=1
        fpr, tpr, thresholds = metrics.roc_curve(tc, pred, pos_label=1) #, metrics.auc(tc, pred)]
        popt, pcov = curve_fit(calibration, pred, target, maxfev=50000)
        fout = calibration(pred, *popt)
        results.append(
            [mean_absolute_error(target, fout),pearsonr(target,fout)[0],spearmanr(target, fout)[0],
             r2_score(target,fout),metrics.auc(fpr, tpr)]
           )
    table = pd.DataFrame(results)
    R[fold-1] = results
    table.columns = metric
    table.index = cells_list
    print(table)

Rf = table.copy()
Rm = R.mean(axis=0)
Rstd = R.std(axis=0)

for i in range(table.shape[0]):
    for j in range(table.shape[1]):
        Rf.iloc[i,j] = str(np.round(Rm[i,j],2)) +" ("+ str(np.round(Rstd[i,j],2))+")"


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Number of test example (827, 256, 256, 3) (827, 4)


/home/dawood/.conda/envs/pt/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


                   MAE   Pearson  Spearman        R2     AUROC
Neuoplatic    1.839836  0.963455  0.948151  0.928242  0.982120
Inflammatory  1.104013  0.935073  0.776273  0.874247  0.884890
Connective    1.664299  0.840770  0.855906  0.706887  0.928787
Epithelial    0.961783  0.975269  0.941927  0.951149  0.992653
Number of test example (749, 256, 256, 3) (749, 4)


/home/dawood/.conda/envs/pt/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


                   MAE   Pearson  Spearman        R2     AUROC
Neuoplatic    1.660397  0.970749  0.933341  0.942353  0.977790
Inflammatory  1.083041  0.945395  0.787624  0.893365  0.892082
Connective    1.707437  0.837058  0.852068  0.700656  0.937933
Epithelial    1.156610  0.968181  0.893327  0.937373  0.989675
Number of test example (775, 256, 256, 3) (775, 4)
                   MAE   Pearson  Spearman        R2     AUROC
Neuoplatic    1.847145  0.961373  0.938676  0.924238  0.979866
Inflammatory  1.040612  0.934387  0.755250  0.873078  0.867140
Connective    1.771188  0.804549  0.840724  0.647298  0.939635
Epithelial    1.125041  0.972401  0.945055  0.945564  0.996457


/home/dawood/.conda/envs/pt/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/dawood/.conda/envs/pt/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [3]:
Rf   

,MAE,Pearson,Spearman,R2,AUROC
Neuoplatic,1.78 (0.09),0.97 (0.0),0.94 (0.01),0.93 (0.01),0.98 (0.0)
Inflammatory,1.08 (0.03),0.94 (0.01),0.77 (0.01),0.88 (0.01),0.88 (0.01)
Connective,1.71 (0.04),0.83 (0.02),0.85 (0.01),0.68 (0.03),0.94 (0.0)
Epithelial,1.08 (0.09),0.97 (0.0),0.93 (0.02),0.94 (0.01),0.99 (0.0)
